This notebook fixes the production of ethanol in the US.

In default ecoinvent 3.4 cutoff, the supply goes like this:
    - 'dewatering of ethanol from biomass, from 95% to 99.7% solution state' (kilogram, US, None)
    - 'ethanol production from maize' (kilogram, US, None)
    - 'market for maize grain' (kilogram, GLO, None)
        - 0.396783400122377 kilogram 'maize grain production' (kilogram, US, None)
        - 0.572106021842784 kilogram 'maize grain production' (kilogram, GLO, None)

However, in our case study this doesn't make sense - we want the maize to be processed into ethanol in the US to be grown in the US. So, we link `'ethanol production from maize' (kilogram, US, None)` directly to US maize grain production.

In [1]:
from brightway2 import *

In [2]:
projects.set_current("LC IMPACT case study")

The broken market

In [8]:
ethanol_from_maize = get_activity(('ecoinvent', 'ce4291110cdbe9be2fc99445b84ddc7e'))
ethanol_from_maize

'ethanol production from maize' (kilogram, US, None)

The input of corn from the global market

In [14]:
ethanol_from_maize = get_activity(('ecoinvent', 'ce4291110cdbe9be2fc99445b84ddc7e'))
for fixme in ethanol_from_maize.technosphere():
    if fixme.input['name'] == 'market for maize grain':
        print(fixme)
        break

Exchange: 3.22588464173796 kilogram 'market for maize grain' (kilogram, GLO, None) to 'ethanol production from maize' (kilogram, US, None)>


In [20]:
fixme

Exchange: 3.22588464173796 kilogram 'maize grain production' (kilogram, RoW_10, None) to 'ethanol production from maize' (kilogram, US, None)>

We can find the correct input (US production of corn):

In [18]:
for usa in fixme.input.technosphere():
    if usa.input['location'] == 'US':
        print(usa)
        break

In [23]:
fixme.input = usa.input
fixme.save()

Check to make sure it worked

In [24]:
ethanol_from_maize = get_activity(('ecoinvent', 'ce4291110cdbe9be2fc99445b84ddc7e'))
for exc in ethanol_from_maize.technosphere():
    print(exc)

Exchange: 0.0360653902946303 kilogram 'market for soda ash, light, crystalline, heptahydrate' (kilogram, GLO, None) to 'ethanol production from maize' (kilogram, US, None)>
Exchange: 13.8834494151494 megajoule 'market for heat, district or industrial, natural gas' (megajoule, RoW_111, None) to 'ethanol production from maize' (kilogram, US, None)>
Exchange: 0.00965475014425753 kilogram 'market for nitrogen fertiliser, as N' (kilogram, GLO, None) to 'ethanol production from maize' (kilogram, US, None)>
Exchange: 3.22588464173796 kilogram 'maize grain production' (kilogram, US, None) to 'ethanol production from maize' (kilogram, US, None)>
Exchange: 0.0322588464173796 ton kilometer 'market for transport, tractor and trailer, agricultural' (ton kilometer, GLO, None) to 'ethanol production from maize' (kilogram, US, None)>
Exchange: -0.00295149089411173 cubic meter 'market for wastewater, from residence' (cubic meter, RoW_164, None) to 'ethanol production from maize' (kilogram, US, None)>
E